# Showcasing functions for implementing privatising of data in a database and creating parallel databases that is iterating over the entire database and creating a new db each one with one entry removed.



In [0]:
#importing torch
import torch

In [0]:
#creating a random tensor filled with 0's or 1's  
db = torch.rand(5000) > 0.5

In [111]:
db

tensor([1, 1, 0,  ..., 1, 1, 0], dtype=torch.uint8)

In [0]:
def create(x):
  return torch.rand(x) > 0.5

In [113]:
create(5000)

tensor([1, 0, 1,  ..., 1, 0, 0], dtype=torch.uint8)

In [114]:
db.shape

torch.Size([5000])

In [0]:
#showing concatenating ops of tensors
T = torch.tensor([1, 2, 3, 4, 5])
i = 2
T = torch.cat([T[:i], T[i+1:]])

In [116]:
x = torch.arange(10)
x

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
value = 5
x = x[x != value]

In [0]:
# a way to remove the value at a index
def remove(tens, index):
  idx = torch.arange(len(tens))  #creating a tensor filled with (0,1,2...)
  tens = tens[idx != index]
  return tens

In [119]:
x = torch.tensor([1, 2, 3, 4])
remove(x, 1)


tensor([1, 3, 4])

In [120]:
db

tensor([1, 1, 0,  ..., 1, 1, 0], dtype=torch.uint8)

In [0]:
#function for creating parallel db that is remove a index from a tensor
def get_parallel_db(db, index):
  return torch.cat((db[:index], db[index + 1 :]))  #way of removing a value at a given index

In [122]:
get_parallel_db(db, 0).shape #shows one element removed since original db contained 5000 elements

torch.Size([4999])

In [0]:
#function for creating a list of paralle dbs each one with one entry removed one by one iterating over the entire db
def get_parallel_dbs(db):
  parallel_dbs = list()
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
  return parallel_dbs  

In [0]:
pdbs = get_parallel_dbs(db)

In [125]:
len(pdbs)

5000

In [0]:
#combined function for creating a db and then fetching required lists of paralle dbs
def create_db_and_paralleldbs(entries):
  db = torch.rand(entries) > 0.5
  pdbs = get_parallel_dbs(db)
  return db, pdbs
  

In [0]:
#creating databases randomly filled with 1's and 0's and then creating parallel databases of length 4999
db, pdbs = create_db_and_paralleldbs(5000)

# Understanding sensitivity of the function


## let's start with sum function 

In [128]:
db

tensor([0, 0, 1,  ..., 1, 0, 1], dtype=torch.uint8)

In [0]:
pdbs

In [0]:
#function for calculating sum over the entire elements of database created
def summation(db):
  return db.sum()

In [138]:
summation(db)

tensor(2482)

In [139]:
summation(pdbs[14])

tensor(2481)

The sensitivity is defined as the maximum change in the query when one instance is removed from the database.


Here , we are going to firstly estimate the total sum of original database , and then iterate over the entire parallel databases generated then calulcate their sum and take absolute difference and find out the maximum of them because we wanna find maximum change occured for the query (here sum).
This is also called L1 norm sensitvity.

In [0]:
total_result = query(db)
def sensitivity(pdbs):
  maximum_distance = 0
  for pdb in pdbs:
    current_result = query(pdb)
    current_distance = torch.abs(current_result - total_result)
    if(current_distance > maximum_distance):
      maximum_distance = current_distance
  return maximum_distance    

In [150]:
sensitivity(pdbs)

tensor(0.0001)

In [0]:
def query(db):
  return db.float().mean()

In [0]:
def L1_senstivity(query, n_entries = 1000):
  db, pdbs = create_db_and_paralleldbs(n_entries)
  maximum_distance = 0
  total_result = query(db)
  for pdb in pdbs:
    current_result = query(pdb)
    current_distance = torch.abs(current_result - total_result)
    if(current_distance > maximum_distance):
      maximum_distance = current_distance
  return maximum_distance    

In [192]:
L1_senstivity(query)

tensor(0.0005)